# Deep Coin

## Project dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import shutil

import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import tensorflow as tf

from keras import applications, optimizers, Input
from keras.models import Sequential, Model, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

%matplotlib inline

### Constants

In [ ]:
TRAIN_DATASET = 'datasets/generated_datasets/only_front/train'
TEST_DATASET = 'datasets/generated_datasets/only_front/test'

## Convolutional Neural Network

In [ ]:
def create_CNN():
    """
    Return the Convolutional Neural Network (CNN) model to classify coin
    """

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

## Train and Test CNN Model

In [ ]:
model = create_CNN()

batch_size = 16
train_datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        rescale=1./255,
        zoom_range=0.1,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DATASET,
        target_size=(64, 64),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        TEST_DATASET,
        target_size=(64, 64),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=200 // batch_size,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=200 // batch_size)

In [ ]:
# model.save('coin_classifier_v1.h5')
print(model.summary())

print(model.evaluate_generator(validation_generator, steps = 32), end='\n\n')
print(train_generator.class_indices)

## Monetary Amount in an image

In [ ]:
def coins_frequency(coins):
    """
    Return the appearance frequency of each specific type of coin.

    arg:
    - coins: list of coin images.
    """

    coins_frequency = {
        '5': 0,
        '10': 0,
        '25': 0,
        '50': 0,
        '100': 0
    }
    
    for coin in coins:
        coin = cv2.resize(coin, (64, 64))
        
        plt.imshow(coin)
        plt.show()
        
        coin = np.reshape(coin, [1, 64, 64, 3])
        coin = np.asarray(coin)
        
        coin_type = model.predict_classes(coin)
        if coin_type[0] == 0:
            print('Moeda: 10 centavos')
            coins_frequency['10'] += 1
        elif coin_type[0] == 1:
            print('Moeda: 1 real')
            coins_frequency['100'] += 1
        elif coin_type[0] == 2:
            print('Moeda: 25 centavos')
            coins_frequency['25'] += 1
        elif coin_type[0] == 3:
            print('Moeda: 5 centavos')
            coins_frequency['5'] += 1
        elif coin_type[0] == 4:
            print('Moeda: 50 centavos')
            coins_frequency['50'] += 1
    
    return coins_frequency

In [ ]:
# model = load_model('coin_classifier_v1.h5')

def main():
    # Coin Detector made with dlib library
    detector = dlib.simple_object_detector("./assets/hog_detector.svm")

    # Show the Histogram Of Oriented Gradient of a coin
    dlib_window = dlib.image_window()
    dlib_window.set_image(detector)
    
    # Detect all coins in an image
    sample_image = cv2.imread('./assets/images/sample1.jpg')[:, :, ::-1]
    gray_image = cv2.cvtColor(sample_image, cv2.COLOR_RGB2GRAY)
    blur_image = cv2.GaussianBlur(gray_image, (3,3), 2)

    coins_regions = detector(blur_image)
    
    # Create list with coins regions extracted from image
    coins_images = []
    for coin_region in coins_regions:
        left, top = coin_region.left(), coin_region.top()
        right, bottom = coin_region.right(), coin_region.bottom()
        
        coin_image = get_image_region(sample_image, left, top, right, bottom)
        coins_images.append(coin_image)
    
    # Compute coins frequency
    coins_repetitions = coins_frequency(coins_images)
    print('Moedas: ', coins_repetitions)
    
    # Show the original image
    plt.imshow(sample_image)
    plt.show()
    
    # Compute the total amount of money
    monetary_amount = 0
    monetary_amount += 5*coins_repetitions['5']
    monetary_amount += 10*coins_repetitions['10']
    monetary_amount += 25*coins_repetitions['25']
    monetary_amount += 50*coins_repetitions['50']
    monetary_amount += 100*coins_repetitions['100']

    print('Total: {:.2f} reais'.format(monetary_amount/100))

In [ ]:
if __name__ == '__main__':
    main()